# Hands-on session on reduced-order models I

In this session, we are going to have a look at how we can use the Proper Orthogonal Decomposition (POD) and the Principal Component Analysis (PCA) to analyse a combustion dataset.

As we'll see, POD and PCA represent the same mathematical technique but they are employed in slightly different ways. POD is generally used for the analysis of time-evolving reacting flows while PCA is used to find a low-dimensional representation of the thermo-chemical space (the manifold).



## Dataset

The dataset comes from the 2D simulation of a pulsating $\mathrm{CH}_4$/$\mathrm{N}_2$ laminar flame. The pulsating behaviour is achieved by modulating the inlet velocity with two different characteristic frequencies:
* simple 10 Hz modulation,
* complex 10, 20, 80 Hz modulation.

The grid contains $n_c = 21134$ cells, and it is stored in the file "grid.vtu".

The dataset contains 3 files. In the "pod" files, the two simulations are arranged in the POD way (the columns contain different timesteps). Each file contains a matrix of size $n \times m$, where $n = n_f n_c = 2 \cdot 21134$. The two features included are the temperature in K and the mass fraction of $\mathrm{OH}$, which is a radical species used to identify the reaction zone.
The number of timesteps is $m = 516$, with a $\Delta t = 5\cdot 10^{-4}$ s.

In the "pca" file, the data of the two simulations is arranged in a single matrix of size $2 n_c m \times n_f = 10820608 \times 9$. The number of timesteps included is $256$ and the features included are the temperature and the species mass fraction of $\mathrm{CH}4$, $\mathrm{CO}$, $\mathrm{CO}_2$, $\mathrm{H}_2$, $\mathrm{H}_2\mathrm{O}$, $\mathrm{N}_2$, $\mathrm{O}_2$, $\mathrm{OH}$

The dataset has been uploaded on zenodo and it can be found here https://zenodo.org/doi/10.5281/zenodo.13490193. We can dowload it directly using ```zenodo_get```, as will see in the next cells.



In [1]:
%%capture
# We install some packages that we'll need later
! pip install zenodo_get
! pip install pyvista
! pip install openmeasure
! git clone http://gitlab.multiscale.utah.edu/common/PCAfold.git
%cd PCAfold
! python -m pip install .
%cd ..

In [ ]:
import zenodo_get
! zenodo_get -d 10.5281/zenodo.13490193

In [ ]:
# Visualise the dataset

import matplotlib.pyplot as plt
import matplotlib as mpl
import pyvista as pv
import numpy as np

# this function can be used to plot a single snapshot
def plot_snapshot(mesh, z, normal, origin, axis, feature, cmap='viridis', filename=''):
    mesh['z'] = z

    plane = mesh.ctp().slice(normal=normal, origin=origin, generate_triangles=True)

    vmin = np.min(plane['z'].min())
    vmax = np.max(plane['z'].max())

    x = plane.points*1000
    tri = plane.faces.reshape((-1,4))[:, 1:]

    fig, ax = plt.subplots(figsize=(2.3, 4))
    levels = 64

    axis_labels = ['x', 'y', 'z']
    fig.subplots_adjust(bottom=0.1, top=.9, left=0.1, right=0.8)

    cs = ax.tricontourf(x[:,axis[0]], x[:,axis[1]], tri, plane['z'], cmap=cmap,
                    levels=levels, vmin=vmin, vmax=vmax)

    ax.set_aspect('equal')
    ax.set_xlabel(f'{axis_labels[axis[0]]} (mm)')
    ax.set_ylabel(f'{axis_labels[axis[1]]} (mm)')


    ax_bounds = ax.get_position().bounds
    cb_ax = fig.add_axes([0.825, ax_bounds[1], 0.05, ax_bounds[3]])
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                        cax=cb_ax, orientation='vertical', label=feature)
    fmt = mpl.ticker.ScalarFormatter(useMathText=True)
    cbar.formatter.set_powerlimits((0, 4))
    cb_ax.yaxis.set_offset_position('left')

    if filename != '':
        fig.savefig(filename, transparent=False, dpi=600, bbox_inches='tight')

    plt.show()

grid = pv.read('grid.vtu') # Load the computational grid

xyz = grid.cell_centers().points
n_cells = xyz.shape[0] # Number of cells

features_pod = ['T', 'OH']

# Select the test case
test_case = 'f10'
D = np.load(f'D_pod_{test_case}.npy')
m = D.shape[1]

# Plot a single snapshot
f_plot = 'T'
i_plot = features_pod.index(f_plot)
snapshot = D[i_plot*n_cells:(i_plot+1)*n_cells, 0]

plot_snapshot(grid, snapshot, (0,1,0), (0,0,0), (0,2),
              features_pod[i_plot], cmap='inferno')


In [ ]:
# We can plot an animation to show the temporal behaviour of the dataset

from matplotlib import rc
rc('animation', html='jshtml')

from matplotlib.animation import FuncAnimation, PillowWriter

def animate(i):
    im1.set_array(D_samples[:n_cells_s,i].reshape(sample_res))
    im2.set_array(D_samples[n_cells_s:,i].reshape(sample_res))
    print(f'Frame {i+1}/{n_frames}', flush=True, end='\r')

    return im1, im2

f_plot = ['T', 'OH']
i_plot = []
for f in f_plot:
    i_plot.append(features_pod.index(f))

gif_step = 5
for i in i_plot:
    for j in range(m//gif_step):
        temp = D[i*n_cells:(i+1)*n_cells, j*gif_step]
        grid[f'f_{i}_timestep_{j}'] = temp

dx = 5e-4
D_samples = np.arange(0, 0.025, dx)
z_samples = np.arange(0, 0.1, dx)

D_samples_grid, z_samples_grid = np.meshgrid(D_samples, z_samples)
xyz_samples = np.zeros((D_samples_grid.size, 3))
xyz_samples[:,0] = D_samples_grid.flatten()
xyz_samples[:,2] = z_samples_grid.flatten()
n_cells_s = xyz_samples.shape[0]

samples = pv.PolyData(xyz_samples)
grid_samples = samples.sample(grid)

D_samples = np.zeros((2*xyz_samples.shape[0], m//gif_step))
for c, i in enumerate(i_plot):
    for j in range(m//gif_step):
        temp = grid_samples[f'f_{i}_timestep_{j}']
        D_samples[c*n_cells_s:(c+1)*n_cells_s,j] = temp

sample_res = D_samples_grid.shape
fps = 50
writergif = PillowWriter(fps=fps)

vmins = []
vmaxs = []

for i in range(len(i_plot)):
    vmins.append(np.min(D_samples[i*n_cells_s:(i+1)*n_cells_s, :]))
    vmaxs.append(np.max(D_samples[i*n_cells_s:(i+1)*n_cells_s, :]))

cmap = 'inferno'

fig, axs = plt.subplots(ncols=2, figsize=(3.5, 3.5), dpi=200)
levels = 64

fig.subplots_adjust(bottom=0., top=1., left=0.275, right=.725, wspace=0.0, hspace=0.05)

im1 = axs[0].imshow(D_samples[:n_cells_s,0].reshape(sample_res), cmap=cmap,
                vmin=vmins[0], vmax=vmaxs[0], origin='lower')

im2 = axs[1].imshow(D_samples[n_cells_s:,0].reshape(sample_res), cmap='viridis',
                vmin=vmins[1], vmax=vmaxs[1], origin='lower')

ax_bounds1 = axs[0].get_position().bounds
cb_ax1 = fig.add_axes([0.2, ax_bounds1[1], 0.025, ax_bounds1[3]])
norm = mpl.colors.Normalize(vmin=vmins[0], vmax=vmaxs[0])
cbar1 = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=cb_ax1, orientation='vertical', label=features_pod[i_plot[0]])
fmt = mpl.ticker.ScalarFormatter(useMathText=True)
cbar1.formatter.set_powerlimits((0, 4))
cb_ax1.yaxis.set_offset_position('right')
cb_ax1.yaxis.set_ticks_position('left')
cb_ax1.yaxis.set_label_position('left')

ax_bounds2 = axs[1].get_position().bounds
cb_ax2 = fig.add_axes([0.775, ax_bounds2[1], 0.025, ax_bounds2[3]])
norm = mpl.colors.Normalize(vmin=vmins[1], vmax=vmaxs[1])
cbar2 = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='viridis'),
                    cax=cb_ax2, orientation='vertical', label=features_pod[i_plot[1]])
fmt = mpl.ticker.ScalarFormatter(useMathText=True)
cbar2.formatter.set_powerlimits((0, 4))
cb_ax2.yaxis.set_offset_position('left')

for i, ax in enumerate(axs):
    if i == 0:
        ax.invert_xaxis()

    ax.set_aspect('equal')
    ax.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, labelleft=False)

n_frames = m//gif_step
anim = FuncAnimation(fig, animate, frames=n_frames, interval=20);

In [ ]:
anim

## Proper Orthogonal Decomposition

In general, the goal of data compression algorithms is to define an encoding function $\mathcal{E}: \mathbb{R}^{n} ↦ \mathbb{R}^{q}$ and a decoding function $\mathcal {D}: \mathbb{R}^{q} ↦ \mathbb{R}^{n}$ such that:

\begin{align}
  \tilde{\mathbf{d}} = \mathcal{D}(\mathcal{E}(\mathbf{d})), \\
  ||\tilde{\mathbf{d}} - \mathbf{d}||_2^2 < \epsilon.
\end{align}

The array $\mathbf{d} \in \mathbb{R}^n$ represent a full-order snapshot of the combustion system (the temperature and velocity field, for example). Each snapshot has dimensions $n = n_c n_f$ where $n_c$ is the number of computational cells and $n_f$ is the number of features.

The encoder compresses the information into a much smaller $q$-dimensional array $\mathbf{a} = \mathcal{E}(\mathbf{d})$. Then, the decoder can be use to reconstruct the full-order snapshot with hopefully minimal error.

If we impose that the encoder and the decoder are linear transformations and orthonormal, we obtain:

\begin{align}
  \mathbf{a} & = \boldsymbol{\Phi}^T \mathbf{d}, \\
  \tilde{\mathbf{d}} & = \boldsymbol{\Phi} \mathbf{a}.
\end{align}

From these equations, the reconstructed snapshot can be written as:

\begin{equation}
  \tilde{\mathbf{d}} = \boldsymbol{\Phi} \boldsymbol{\Phi}^T \mathbf{d}.
\end{equation}

The matrix $\boldsymbol{\Phi} \boldsymbol{\Phi}^T$ has to be close to the identity matrix if we want to have a low reconstruction error.

To find the autoencoding matrix, we need to collect the data and build the data matrix $\mathbf{D} \in \mathbb{R}^{n \times m}$:

\begin{equation}
    \mathbf{D}(\mathbf{r}, t) = \begin{bmatrix}
        \vert  & \vert & & \vert \\
        \mathbf{d}_{1} & \mathbf{d}_{2} & \cdots & \mathbf{d}_{m} \\
        \vert  & \vert & & \vert \\
    \end{bmatrix}  .
\end{equation}

Each column of $\mathbf{D}$ is a snapshot of the system at time $t = i\Delta t$.

The POD objective function can be written as:

\begin{align}
\mathcal{L}(\boldsymbol{\Phi}) & = ||\mathbf{D} - \tilde{\mathbf{D}}||_2^2 =  ||\mathbf{D} - \boldsymbol{\Phi} \boldsymbol{\Phi}^T\mathbf{D}||_2^2 \\
\mathrm{s.t.} & \ \ \ \boldsymbol{\Phi}^T \boldsymbol{\Phi} = \mathbf{I}.
\end{align}

The solution of this optimisation problem correspond to the eigenvalue problem:

\begin{equation}
  \mathbf{C} = \boldsymbol{\Phi} \mathbf{L} \boldsymbol{\Phi}^T,
\end{equation}

where $\mathbf{C} = \mathbf{D}\mathbf{D}^T$ is the spatial correlation matrix and $\mathbf{L}$ is a diagonal matrix containing the eigenvectors of $\mathbf{C}$.

Now we can write the compressed representation of $\mathbf{D}$ as:

\begin{equation}
\mathbf{A} = \boldsymbol{\Phi}^T \mathbf{D}.
\end{equation}

We can also factorise $\mathbf{A} = \boldsymbol{\Sigma} \boldsymbol{\Psi}^T$, where $\boldsymbol{\Sigma}$ is a diagonal matrix that contains the $l_2$ norm of the rows of $\mathbf{A}$.

This means that the reconstructed data matrix can be factorised as:

\begin{equation}
\tilde{\mathbf{D}}(\mathbf{r}, t) = \boldsymbol{\Phi}(\mathbf{r}) \boldsymbol{\Sigma} \boldsymbol{\Psi}(t)^T.
\end{equation}

This equation represents the truncated singular value decomposition of $\mathbf{D}$. It is worth noting also the fact that the POD decomposes the spatial information (contained in $\boldsymbol{\Phi}$) from the temporal information (contained in $\boldsymbol{\Psi}$).

The matrix $\boldsymbol{\Psi}$ contains the temporal coefficients of the POD modes, and it can also be computed from the eigendecomposition of the temporal correlation matrix $\mathbf{K} = \mathbf{D}^T\mathbf{D}$:

\begin{equation}
 \mathbf{K} = \boldsymbol{\Psi} \mathbf{L} \boldsymbol{\Psi}^T.
\end{equation}

So, there are 3 ways to compute the POD decomposition:


*   Eigendecomposition of the spatial correlation matrix (classic POD).
*   Eigendecomposition of the temporal correlation matrix (snapshot POD).
*   Singular value decomposition.

### Data preprocessing

The POD can only be applied to centered matrices (i.e. matrices in which the mean along the rows is zero). If the matrix is not centered, we can center it by removing the mean:

\begin{equation}
\mathbf{D}_0 = \mathbf{D} - \mathbf{D}_{\mu}
\end{equation}

Each column of the matrix $\mathbf{D}_{\mu}$ contains the temporal mean $\mathbf{d}_{\mu}$ computed as:

\begin{equation}
  d_{\mu_i} = \frac{1}{m}\sum_{j=1}^{m} d_{i,j} \ \ \ i = 1, \dots, n.
\end{equation}

If the data matrix contains multiple features with different units of measurements or different scales, we also need to normalise the dataset:

\begin{equation}
    \mathbf{D}_0 = \mathbf{D}_{s}^{-1}(\mathbf{D} - \mathbf{D}_\mu).
\end{equation}

$\mathbf{D}_s$ is a $n \times n $ diagonal matrix computed as:

\begin{equation}
    \mathbf{D}_s = \begin{bmatrix}
        s_1 \mathbf{I} & \mathbf{0} & \dots & \mathbf{0} \\
        \mathbf{0} & s_2 \mathbf{I} & \dots & \mathbf{0} \\
        \vdots & \vdots & \ddots & \vdots \\
        \mathbf{0} & \mathbf{0} & \dots & s_{n_f} \mathbf{I}
    \end{bmatrix}
\end{equation}

where $\mathbf{I}$ is the $n_c \times n_c$ identity matrix. The scaling coefficient $s_i$ is generally chosen to be the standard deviation of the $i$-th feature, so that the variance of each scaled feature is equal to 1.

The reconstructed dataset is then:

\begin{equation}
  \tilde{\mathbf{D}} = \mathbf{D}_{\mu} + \mathbf{D}_{s}(\boldsymbol{\Phi}\boldsymbol{\Sigma}\boldsymbol{\Psi}^T)
\end{equation}

or, in vector notation:

\begin{equation}
  \tilde{\mathbf{d}}(t) = \mathbf{d}_{\mu} + \mathbf{D}_{s}( \sum_{i=1}^{q} \boldsymbol{\phi}_i \sigma_i \psi_i(t))
\end{equation}

### Truncation

We have seen that POD compresses the information into the first $q$ modes, but how do we choose $q$? One way is to select a threshold of the retained variance (or energy) equal to a high percentage of the original variance, such as $90 \%$, $95 \%$ or $99 \%$.

The amount of variance retained in each mode is indicated by the respective eigenvalue. The relative variance can be computed as:

\begin{equation}
  V(i) = \frac{l_i}{\sum_{i=1}^{m} l_i},
\end{equation}

while the relative cumulative variance is:
\begin{equation}
  V(q) = \frac{\sum_{i=1}^{q} l_i}{\sum_{i=1}^{m} l_i}.
\end{equation}

## References:
* [Elements of Dimensionality Reduction and Manifold Learning](https://link.springer.com/book/10.1007/978-3-031-10602-6) (book).
* [Linear and nonlinear dimensionality reduction from fluid mechanics to machine learning](https://iopscience.iop.org/article/10.1088/1361-6501/acaffe/meta) (paper).
* [On the hidden beauty of the proper orthogonal decomposition](https://link.springer.com/article/10.1007/BF00271473) (paper).


In [19]:
# Exercise 1:
# - Compute the eigendecomposition of the temporal correlation matrix
# of the D_f10 dataset (considering only the temperature)

# - Plot the relative variance of the first 10 eigenvalues
# - Plot their cumulative relative energy

# Load the dataset and select only the temperature
feature = 'T'
i_f = features_pod.index(feature)

D = np.load('D_pod_f10.npy')[i_f*n_cells:(i_f+1)*n_cells, :]
m = D.shape[1]

# Tips:
# - Use the function print() to print information to the cell's output
# - To compute the mean, you can use the function np.mean()
# - The arrays can be manipulated using numpy slicing (https://numpy.org/doc/stable/user/basics.indexing.html)
# - Matrices can be multiplied using the function np.dot()
# - To compute the eigendecomposition, use the function np.linalg.eig()
# - To plot, use the function plt.plot() or plt.scatter()


In [20]:
# Exercise 2:
# - Compute the phase angle alpha of the modulating sinusoid.
# - Plot the distribution of the temporal coefficients of the first 2 modes,
#   colored by the phase angle.

# Tips:
# - The phase angle is alpha = 2*pi*(time/period % 1) (% is the reminder operator)
# - To plot, use plt.scatter(x, y, c=alpha)


In [21]:
# Exercise 3:
# - Compute the spatial modes
# - Plot the 1st and 2nd modes

# Tips:
# - Compute the l2 norm using np.linalg.norm()
# - Use the function plot_snapshot() to plot the modes

# Plot them
# plot_snapshot(grid, mode1, (0,1,0), (0,0,0), (0,2), '', cmap='inferno')



In [22]:
# Exercise 4:
# - Reconstruct the dataset using only 4 modes
# - Compare the original and reconstructed last snapshot

# Tips:
# - Use the function np.linalg.multi_dot() to multiply 3 or more matrices

# plot_snapshot(grid, original, (0,1,0), (0,0,0), (0,2), 'T', cmap='inferno')
# plot_snapshot(grid, reconstructed, (0,1,0), (0,0,0), (0,2), 'T', cmap='inferno')
# plot_snapshot(grid, difference, (0,1,0), (0,0,0), (0,2), 'T', cmap='inferno')


In [23]:
# Bonus exercise:
# - Compute the POD on the matrix containing both the temperature and OH
# - Plot the cumulative energy of the eigenvalues
# - Plot the first temperature and OH POD modes

D = np.load('D_pod_f10.npy')
m = D.shape[1]


## Principal Component Analysis

As a mathematical technique, PCA is exactly the same as POD. However, in the combustion community this technique is called POD when the goal is to analyse the temporal behaviour of dynamical systems, while it is called PCA when it is used to find a low-dimensional projection of the thermo-chemical state (temperature and species concentrations).

Finding a low-dimensional representation of the thermo-chemical state is very important in combustion, because reacting simulations can involve hundreds of species and thousands of reactions.

In this case, the data matrix $\mathbf{D} \in \mathbb{R}^{s \times n_f}$ is arranged as:

\begin{equation}
    \mathbf{D} = \begin{bmatrix}
        - & \mathbf{d}_{1} & - \\
        - & \mathbf{d}_{2} & -  \\
          & \vdots & \\
        - & \mathbf{d}_{s} & - \\
    \end{bmatrix}  .
\end{equation}

Each row represents the position of a point in the thermo-chimical space. By diagonalising the row-wise correlation matrix $\mathbf{K}$ we obtain the principal components (PCs):

\begin{equation}
\mathbf{K} = \boldsymbol{\Psi}\mathbf{L}\boldsymbol{\Psi}^T
\end{equation}

The matrix $\boldsymbol{\Psi}$ transform the data from its high-dimensional representation $\mathbf{D}$ to the low-dimensional representation $\mathbf{Z}$:

\begin{equation}
\mathbf{Z} = \mathbf{D} \boldsymbol{\Psi}.
\end{equation}

The rows of $\mathbf{Z}$ are called the principal scores.

### Data preprocessing

As for POD, the dataset needs to be centered and scaled before applying PCA. However, since the data is arranged in a different way, this process is slightly different:

\begin{equation}
  \mathbf{D}_0 = (\mathbf{D} - \mathbf{D}_{\mu})\mathbf{D}^{-1}_{s}.
\end{equation}

The matrix $\mathbf{D}_{\mu}$ contains the column-wise average while $\mathbf{D}_{\mu} \in \mathbb{R}^{n_f \times n_f}$ is a diagonal matrix containing the standard deviation of each feature.

## References:
* [Principal component analysis of turbulent combustion data: Data pre-processing and manifold sensitivity](https://doi.org/10.1016/j.combustflame.2012.09.016) (paper).
* [Cost function for low-dimensional manifold topology assessment](https://www.nature.com/articles/s41598-022-18655-1) (paper).

In [26]:
# Exercise 1:
# - Load the file "D_pca.npy"
# - Plot the O2 vs CO2 concentrations and color them by the temperature
#   (plot only 10000 random points)

D = np.load('D_pca.npy')
features_pca = ['T', 'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2', 'O2', 'OH']

# This is focus on the reactive region
i_OH = features_pca.index('OH')
mask = D[:, i_OH] > 1e-4
D = D[mask, :]

# Use random_int to plot the points
rng = np.random.default_rng(12312043)
random_int = rng.integers(D.shape[0], size=10000)


In [25]:
# Exercise 2:
# - Apply PCA to the dataset
# - Plot the 1st and 2nd scores and color them by the temperature

# Tips:
# - Use np.std() to compute the standard deviation


In [24]:
# Exercise 3:
# - Visualise the weights of the first 3 PCs

# Tips:
# - Use plt.bar() to create a bar plot of the weights


In [ ]:
# Exercise 4:
# - Plot the distribution of the relative variance and cumulative variance

In [ ]:
# Bonus: 
# - Do PCA using PCAFold.
# - Select different scaling methods.

from PCAfold.reduction import PCA

# Tips:
# Find the PCA documentation here: https://pcafold.readthedocs.io/en/latest/user/data-reduction.html

## Reduced-order model

Another important feature of POD is that it allows us to easily build ROMs of combustion systems. A ROM exploits data compression to reduce the complexity of the full-order system and to build a regression model that can instantly predict the system's state.

In our example, we want to build a model that predicts the system's state given the phase angle:

\begin{equation}
  \mathbf{d} = f(\alpha).
\end{equation}

The function $f: \mathbb{R} \mapsto \mathbb{R}^{n}$ has very high dimensionality, so it is complex to represent. However, we can first project the data in the low-dimensional manifold, and then build the regression model:

\begin{align}
  & a_i = g_i(\alpha) \ \ \ i=1, \dots, q \\
  & \mathbf{d} = \sum_{i=1}^{q} \boldsymbol{\phi}_i(\mathbf{r}) \ a_i(\alpha).
\end{align}

The functions $g_i: \mathbb{R} \mapsto \mathbb{R}$ are much simpler to estimate than $f$.

This framework is very similar to the KKL expansion, which is regarded as the generalisation of POD for continuuos functions. This expansion states that a centered stochastic process $D_t$ can be decomposed as:

\begin{equation}
  D_t = \sum_{i=1}^{\infty} \Phi_i \ a_i(t),
\end{equation}

where $a_i(t)$ are orthogonal functions.

To estimate the functions $g_i$ we can employ different methods from linear regression to neural networks. In our group, we employ the Gaussian Process Regression (GPR), which is a Bayesian method that let us estimate also the uncertainty in the estimation:

\begin{equation}
  g_i(\alpha^*) \sim \mathcal{N}(\bar{g}_i(\alpha^*), \mathrm{var}(g_i(\alpha^*))).
\end{equation}

You can find some more information of GPR and the GPR-based ROM in the documentation of [OpenMEASURE](https://github.com/burn-research/OpenMEASURE).

## References:
* [Application of reduced-order models based on PCA & Kriging for the development of digital twins of reacting flow applications](https://doi.org/10.1016/j.compchemeng.2018.09.022) (paper).
* [Parameter Estimation Using a Gaussian Process Regression-Based Reduced-Order Model and Sparse Sensing: Application to a Methane/Air Lifted Jet Flame](https://link.springer.com/article/10.1007/s10494-023-00446-x) (paper).


In [27]:
# Exercise 1:
# - Compute the spatial modes matrix Phi
# - Plot the first 4 functions g_i (the projection of D) as a function of alpha

# Load the matrix with only the temperature data
# Skip the first 196 timesteps (not periodic)

n_init = 196
D = np.load('D_pod_fcomplex.npy')[i_f*n_cells:(i_f+1)*n_cells, n_init:]
m = D.shape[1]

# Tips:
# - You can compute Phi using np.svd() with the option full_matrices=False


In [15]:
# Exercise 2:
# - Use openmeasure to build a ROM that predicts a from alpha
# - Predict the solution and the uncertainty for 100 points between 0-2pi

from openmeasure.gpr import GPR

# Choose 32 snapshots randomly
rng = np.random.default_rng(122213423)
i_train = np.arange(m)
rng.shuffle(i_train)
i_train = i_train[:32]

# alpha_train = alpha[i_train, np.newaxis]
# D_train = D[:, i_train]

xyz = grid.cell_centers().points

# Tips:
# - Check https://github.com/burn-research/OpenMEASURE/blob/master/docs/gpr_doc.ipynb
#   for the instructions on how to use GPR()

# - Create the gpr model using the class GPR() and supplying
#   the dataset, the number of features, the grid points xyz and alpha

# - Compute the POD using gpr.fit() with the option scaleX_type='none'
# - Train the model using gpr.train()
# - Predict the solution using gpr.predict()
# - Use np.linspace() to create 100 equidistant points


In [28]:
# Exercise 4:
# - Plot the predictions a and their uncertainty

# Tips:
# - Use plt.fill_between() to plot the uncertainty

In [ ]:
# Exercise 5:
# - Reconstruct the dataset D_test from the prediction A_test

# Tips:
# - Use gpr.reconstruct()

In [ ]:
# Bonus: animate the prediction